In [1]:
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")
from ronek import env
from ronek.plasmas import ArgonCR
import ronek.backend as bkd

In [3]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 8,
  "epsilon": 1e-10,
  "floatx": "float64"
}
env.set(**env_opts)

In [4]:
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/CR_Ar/database"

In [5]:
model = ArgonCR(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  use_coll_int_fit=True,
  use_pe=True
)

In [6]:
sol_ref = "/home/zanardi/Workspace/AirDatabase/ThermoChemModels/MT/run/CR_Ar/output_box"

In [7]:
columns = "t X_em X_Ar X_Arp Th Te rho p n E".split()
box = np.loadtxt(sol_ref + "/box.dat")
box = pd.DataFrame(box, columns=columns)
box

,t,X_em,X_Ar,X_Arp,Th,Te,rho,p,n,E
0,0.000000e+00,1.000000e-14,1.000000,1.000000e-14,30000.000000,300.000000,0.016015,100000.000000,2.414324e+23,9.365946e+06
1,1.000000e-10,1.925875e-09,1.000000,1.925875e-09,29999.987847,0.025542,0.016015,99999.959488,2.414324e+23,9.365946e+06
2,2.015000e-10,3.898112e-09,1.000000,3.898112e-09,29999.975513,0.053376,0.016015,99999.918376,2.414324e+23,9.365946e+06
3,3.045225e-10,5.917557e-09,1.000000,5.917557e-09,29999.962993,0.086507,0.016015,99999.876644,2.414324e+23,9.365946e+06
4,4.090903e-10,7.985314e-09,1.000000,7.985314e-09,29999.950284,0.124765,0.016015,99999.834280,2.414324e+23,9.365946e+06
...,...,...,...,...,...,...,...,...,...,...
797,9.491642e-04,1.203514e-01,0.759297,1.203514e-01,11649.027101,11649.027101,0.016015,44142.732052,2.744646e+23,9.365946e+06
798,9.634018e-04,1.203514e-01,0.759297,1.203514e-01,11649.027101,11649.027101,0.016015,44142.732052,2.744646e+23,9.365946e+06
799,9.778529e-04,1.203514e-01,0.759297,1.203514e-01,11649.027101,11649.027101,0.016015,44142.732052,2.744646e+23,9.365946e+06
800,9.925208e-04,1.203514e-01,0.759297,1.203514e-01,11649.027101,11649.027101,0.016015,44142.732052,2.744646e+23,9.365946e+06


In [8]:
t = box["t"].values
rho, Th0, Te0 = [box[k][0] for k in ("rho", "Th", "Te")]

In [9]:
def read_pop(name):
  ni_gi  = np.loadtxt(sol_ref + f"/pop_{name}.dat", comments="&", skiprows=2)[:,1]
  levels = model.mix.species[name].lev
  nb_pts = len(ni_gi) / len(levels["g"])
  ni_gi  = np.stack(np.split(ni_gi, nb_pts, axis=0), axis=0)
  return ni_gi * levels["g"].numpy()

In [10]:
n = {k: read_pop(k) for k in ("Ar", "Arp")}
n["em"] = (box["X_em"] * box["n"]).values.reshape(-1,1)
n = np.hstack([n[k] for k in model.species_order]).T
w_true = model.mix.get_w(rho, bkd.to_torch(n)).numpy()
T_true = box[["Th", "Te"]].values.T

In [11]:
y0 = np.zeros(model.nb_eqs)
y0[:model.mix.nb_comp] = w_true[:,0]
y0[-2] = Th0
y0[-1] = Te0
y0 = bkd.to_torch(y0)

In [11]:
y = model.solve(t, y0, rho)
w_pred = y[:model.mix.nb_comp]
T_pred = y[model.mix.nb_comp:]

In [ ]:
err = []
for i in range(2):
  ierr = np.mean(np.abs((T_true[i] - T_pred[i])/T_true[i]))
  err.append(ierr)
  plt.loglog(t[1:], T_true[i][1:])
  plt.loglog(t[1:], T_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
print("T")
print(np.mean(err))
plt.show()
plt.close()

In [ ]:
err = []
for i in range(2):
  ierr = np.mean(np.abs((T_true[i] - T_pred[i])/T_true[i]))
  err.append(ierr)
  plt.semilogx(t[1:], T_true[i][1:])
  plt.semilogx(t[1:], T_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
print("T")
print(np.mean(err))
plt.show()
plt.close()

In [ ]:
for s in model.mix.species.values():
  print(s.name)
  err = []
  for i in s.indices:
    ierr = np.mean(np.abs((w_true[i] - w_pred[i])/w_true[i]))
    err.append(ierr)
    plt.loglog(t[1:], w_true[i][1:])
    plt.loglog(t[1:], w_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  print(np.mean(err))
  plt.show()
  plt.close()